https://stepik.org/lesson/240241/step/4?unit=214000


https://github.com/ivanov-v-v/rosalind-mipt-2019/blob/master/06%20%E2%80%94%20ba2d/main.py

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from scipy import stats

In [2]:
alphabet = list('ACGT')
letter_to_id = {letter: idx for idx, letter in enumerate(alphabet)}

In [3]:
def profile_random_kmer(text, k, profile_matrix):
    
    """
    output:
        profile-randomly generated kmer in text for given profile matrix
    """
    
    p = []
    
    for i in range(len(text)-k+1):
        kmer = text[i:i+k]
        indexes = [letter_to_id[char] for char in kmer]
        
        string_proba = np.choose(indexes, profile_matrix).prod()
        
        p.append(string_proba)
    
    p = np.array(p) / sum(p)
    
    start = np.random.choice(a=np.arange(len(text)-k+1), p=p)
    rand_kmer = text[start:start+k]
    
    return np.array(list(rand_kmer))

In [4]:
def most_probable_kmer(text, k, profile_matrix):
    
    """
    output:
        most probable kmer in text for given profile matrix
    """
    
    best_proba = -np.inf
    most_proba_kmer = None
    
    for i in range(len(text)-k+1):
        kmer = text[i:i+k]
        indexes = [letter_to_id[char] for char in kmer]
        
        string_proba = np.choose(indexes, profile_matrix).prod()
        
        if string_proba > best_proba:
            best_proba = string_proba
            most_proba_kmer = kmer
    
    return np.array(list(most_proba_kmer))

In [5]:
def get_profile_with_pseudocounts(motifs):
    
    """
    input:
        motifs: np.array
    
    output:
        profile matrix
    """
    
    counter = np.apply_along_axis(Counter, 0, motifs)
    counter = [dict(c) for c in counter]
    
    count_motifs = pd.DataFrame(counter, columns=list('ACGT')).fillna(0).T.to_numpy()
    
    # Laplace’s Rule of Succession
    count_motifs = count_motifs + 1
    
    # Plus 4, since for each letter in the alphabet (ACGT) we add 1
    profile = count_motifs / (len(motifs) + 4)
    
    return profile

In [6]:
def get_scores(motifs, t):
    
    """
    input:
        motifs: np.array
    
    output:
        score of given motifs matrix
    """
    
    modes, counts = stats.mode(motifs)
    scores = t - counts
    
    return scores.sum()

In [7]:
def get_motifs_profile_dna(profile, dna, k):
    motifs = []
    for text in dna:
        motifs.append(most_probable_kmer(text, k, profile))
    
    return np.row_stack(motifs)

In [8]:
dna = [
    "ttaccttaac",
"gatgtctgtc",
    "acggcgttag",
    "ccctaacgag",
"cgtcagaggt"
]

In [9]:
dna = [text.upper() for text in dna]

In [10]:
dna

['TTACCTTAAC', 'GATGTCTGTC', 'ACGGCGTTAG', 'CCCTAACGAG', 'CGTCAGAGGT']

In [11]:
k = 4

In [12]:
profile = np.array([
    [4/5, 0, 0, 1/5],
    [0, 3/5, 1/5, 0],
    [1/5, 1/5, 4/5, 0],
    [0, 1/5, 0, 4/5]
])
profile

array([[0.8, 0. , 0. , 0.2],
       [0. , 0.6, 0.2, 0. ],
       [0.2, 0.2, 0.8, 0. ],
       [0. , 0.2, 0. , 0.8]])

In [13]:
get_motifs_profile_dna(profile, dna, k)

array([['A', 'C', 'C', 'T'],
       ['A', 'T', 'G', 'T'],
       ['G', 'C', 'G', 'T'],
       ['A', 'C', 'G', 'A'],
       ['A', 'G', 'G', 'T']], dtype='<U1')

In [14]:
e = [4,5,7,6,6]

In [15]:
from tqdm import tqdm

In [16]:
def gibbs_sampler(dna, k, t, N):
    # including pseudocounts
    
    dna_arr = np.array([list(text) for text in dna])
    
    # random index selection
    idxs = np.random.randint(0, dna_arr.shape[1]-k+1, t)[:, None]
    idxs = idxs + np.arange(k)
    
    # use indexes to randomly select the initial motifs
    best_motifs = np.take_along_axis(dna_arr, idxs, axis=1)
    best_score = np.inf
    
    
    motifs_list = best_motifs.copy()
    for j in range(N):
        
        i = np.random.randint(0, t)
        
        all_motifs_but_i = np.delete(motifs_list, i, axis=0)
        
        profile = get_profile_with_pseudocounts(all_motifs_but_i)
        motif_i = profile_random_kmer(dna[i], k, profile)
        
        motifs_list = np.insert(all_motifs_but_i, i, motif_i, axis=0)
        
            
        score_motifs_list = get_scores(motifs_list, t)
        
        if score_motifs_list < best_score:
            best_score = score_motifs_list
            best_motifs = motifs_list.copy()
    
    best_motifs = [''.join(motif_arr.tolist()) for motif_arr in best_motifs]
    
    return best_motifs, best_score

In [22]:
def find_best_approx(dna, k, t, N):
    
    best_motifs = None
    best_score = np.inf
    
    curr_score = np.inf
    for i in tqdm(range(10)):
        curr_motifs, curr_score = gibbs_sampler(dna, k, t, N)
        
        if curr_score < best_score:
            best_score = curr_score
            best_motifs = curr_motifs
    
    
    return best_motifs

In [23]:
k, t, N = 8, 5, 100

In [24]:
dna = [
    'CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA',
    'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG',
    'TAGTACCGAGACCGAAAGAAGTATACAGGCGT',
    'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC',
    'AATCCACCAGCTCCACGTGCAATGTTGGCCTA'
]

In [25]:
def main():
    
    file = open('gibbs.txt', 'r')
    
    k, t, N = list(map(int, next(file).split()))
    
    #print(k, t)
    
    dna = []
    for string in file:
        dna.append(string.strip())
        
    #print(dna)
    
    
    print("\n".join(find_best_approx(dna, k, t, N)))

    file.close()

In [26]:
if __name__ == "__main__":
    main()

100%|██████████| 10/10 [04:13<00:00, 25.33s/it]

TGACGTCCACCGGCG
GTAAGCGCACCGGGG
TTACCCTTACCGGGG
AGAAGTTCCTCGGGG
ATAAGTTTTATGGGG
GCAAGTTTACCGGGT
CCAAGTTTCGAGGGG
TCCTGTTTACCGGGG
TTAAGTTGCTCGGGG
TTAAACATACCGGGG
GCAAGTTTAGGAGGG
GTAAGGAAACCGGGG
GTAAGTTTACACAGG
ATTAGTTTACCGGGG
CCCCTTTTACCGGGG
TGAAGTGAGCCGGGG
TAAAGTCGTCCGGGG
GAAAGTTTACCGGAC
CAAAGTTTACCAATG
CCAAGTTTACCGTCA


In [109]:
ACGTCCACCGGCGTC
AAGCGCACCGGGGTG
ACCCTTACCGGGGTG
AAGTTCCTCGGGGTG
AAGTTTTATGGGGTG
AAGTTTACCGGGTGC
AAGTTTCGAGGGGTG
CTGTTTACCGGGGTA
AAGTTGCTCGGGGTG
AAACATACCGGGGTG
AAGTTTAGGAGGGTG
AAGGAAACCGGGGTG
AAGTTTACACAGGTG
TAGTTTACCGGGGAT
CCTTTTACCGGGGTG
AAGTGAGCCGGGGTG
AAGTCGTCCGGGGTG
AAGTTTACCGGACAG
AAGTTTACCAATGTG
AAGTTTACCGTCATG

ACCGGCCA
AGGTGCCA
CCGAGACC
AGGTGCAC
ACGTGCAA


In [14]:
np.random.randint(0, 5, 7)

array([0, 2, 0, 4, 0, 4, 1])

In [ ]:
np.choice(np.arange())

In [15]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [17]:
text = 'GAAGCACTGAGTTACCAGGTAC'

In [20]:
k, len(text)

(4, 22)

In [32]:
np.arange(len(text)-k+1)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [27]:
from scipy.special import softmax

In [40]:
p = np.random.rand(len(text)-k+1)

In [42]:
(p / p.sum()).sum()

0.9999999999999998

In [39]:
np.random.choice(a=np.arange(len(text)-k+1), p=p)

18

In [63]:
np.random.randint(0, 5)

3

In [67]:
see = np.random.randint(0, 10, size=(4,5))
see

array([[0, 9, 6, 6, 6],
       [4, 5, 5, 6, 9],
       [2, 3, 3, 1, 3],
       [9, 6, 9, 5, 1]])

In [72]:
see_del = np.delete(see, 2, axis=0)
see_del

array([[0, 9, 6, 6, 6],
       [4, 5, 5, 6, 9],
       [9, 6, 9, 5, 1]])

In [73]:
see2 = np.zeros(5)
see2

array([0., 0., 0., 0., 0.])

In [75]:
np.insert(see_del, 2, see2, axis=0)

array([[0, 9, 6, 6, 6],
       [4, 5, 5, 6, 9],
       [0, 0, 0, 0, 0],
       [9, 6, 9, 5, 1]])

In [14]:
def find_best_approx(dna, k, t, n_times=1000):
    
    best_motifs = None
    best_score = np.inf
    
    curr_score = np.inf
    for i in tqdm(range(n_times)):
        curr_motifs, curr_score = randomized_greedy_motif_search(dna, k, t, curr_score)
        
        if curr_score < best_score:
            best_score = curr_score
            best_motifs = curr_motifs
    
    
    return best_motifs

In [15]:
from tqdm import tqdm

In [21]:
def main():
    
    file = open('rosalind_ba2f.txt', 'r')
    
    k, t = list(map(int, next(file).split()))
    
    print(k, t)
    
    dna = []
    for string in file:
        dna.append(string.strip())
        
    print(dna)
    
    
    print("\n".join(find_best_approx(dna, k, t)))

    file.close()

In [22]:
if __name__ == "__main__":
    main()

  0%|          | 0/1000 [00:00<?, ?it/s]

15 20
['GAAGCACTGAGTTACCAGGTACACAGGTCTCAGTGCACGGACTGCACCGGGCTCAGCCCTGTAGGCGTGAACCTTCCAACATCGACTGGGGATGATCTAATCTTCGCATCGTACGCACGTCAATAGCATAACGTCGGACACGTACCCTAGGCAACAACCACTTTTGACCTTCCTTCCGGCCGCAAGGCTCAACTTAGTGGAAGCACTGAGTTAC', 'CAGGTACACAGGTCTCAGTGCACGGACTGCACCGGGCTCAGCCCTGTAGGCGTGAACCTTCCAACATCGACTGGGGATGATCTAATCTTCGCATCGTACGCACGTCAATAGCATAACGTCGGACACGCCCCAATAGTCAGCGTACCCTAGGCAACAACCACTTTTGACCTTCCTTCCGGCCGCAAGGCTCAACTTAGTGGAAGCACTGAGTTAC', 'ATACTCTTAATCATTATAACATGAGACAGCCTATCACACGTACACGATCGCCAATGTTCCCCCGCTCGAATTAGACCGGTTTGGAATTGAGTTCCGTGGGGGTATGCGCCGCACAAACTGCTATAGTAAAACCACCCTTTTAGTCAGCGTATTGACATATTACAAGCGGCGTTTACTCCTTAGAAAGGCAATAAACGAACAAAACAGTACCCGC', 'ACTGGCTTCCAGAAAACACCAGGCCCCAATTTCGGCGGATGGATTGTTCACAATGCTTGATCGTGCGTGCGCCATATTGCTGCCGACCGCGGCCGGATATACCGGTGCCTTTCCTAGAAATTGTCGGATAATACTCCTGCCCCAATGAGTCAGCGTATACAGTACGGGACCCATGGTCAAGAACGTTAAGTATTCGTAGGGCGCGACACACAGT', 'GCCCAGGCCCTTCCCAGCTTAGCAGCGTCTGTCGATGTTCCTGCTGCGATGGCTCGCCGAGACTACAAGAGTGGAGCGAGCCGAAATATCAGAAGCACCGAGCATGAGCTCGGGCCGCGA

100%|██████████| 1000/1000 [02:41<00:00,  6.20it/s]

ACCAGGTACACAGGT
CCCCAATAGTCAGCG
CCCTTTTAGTCAGCG
CCCAATGAGTCAGCG
CCCAGCTTAGCAGCG
AGCAGCTAGTCAGCA
CCCAGCTAGATCGCG
CCCACTAAGTCAGCG
CCGGACTAGTCAGCG
CCCAGCGCATCAGCG
CCCAGCTATGTAGCG
CCCAGCTAGTAGTCG
TCCAGCTAGTCAGTT
CCCAGGGGGTCAGCG
TCGAAGATGTCAGCG
CCCAGGGTGTCAGCG
CCCAGCTAGTCACGA
CCCAGCAGCTCAGCG
CCCAGCTAGTCTATG
CAGGGCTAGTCAGCG


In [18]:
CATGGGGAAAACTGA
CCTCTCGATCACCGA
CCTATAGATCACCGA
CCGATTGATCACCGA
CCTTGTGCAGACCGA
CCTTGCCTTCACCGA
CCTTGTTGCCACCGA
ACTTGTGATCACCTT
CCTTGTGATCAATTA
CCTTGTGATCTGTGA
CCTTGTGATCACTCC
AACTGTGATCACCGA
CCTTAGTATCACCGA
CCTTGTGAAATCCGA
CCTTGTCGCCACCGA
TGTTGTGATCACCGC
CACCGTGATCACCGA
CCTTGGTTTCACCGA
CCTTTGCATCACCGA
CCTTGTGATTTACGA

NameError: name 'CATGGGGAAAACTGA' is not defined

In [ ]:
ACGGAGATTTCTGGC
ACGGAGATTTCTGGC
CCCGCTGATTGTCTG
GCGACTGGTTCCTTA
CCGTAAATATCAGGA
CAGGACGTTTAGGTA
ACCTACGTTTCTGGA
GCGGACGGTGCCTGG
TGGTATAATTCTGTG
TCCGTAGGTTGTGGA
CAGGATGAATCTGGA
TGGGCGGATTCATGA
GGGTAAGATGCCGCA
CCGTACTGTTATGGT
AACGAGGATTGTTCA
CCCTTTTTTTCAGCA
ATGGAGTTTGACTCA
TCGGACATTCCTGGA
ATGGAGTTTCCATCC
TCGAAGGTTTCTTGG